In [4]:
import numpy as np
from scipy.optimize import linprog
from scipy.linalg import eig
from itertools import permutations
import matplotlib.pyplot as plt

def random_doubly_stochastic(n, num_terms=100):
    """Create a random doubly stochastic matrix via convex combinations of permutation matrices."""
    perms = [np.array(p) for p in permutations(range(n))]
    matrices = [np.eye(n)[p] for p in perms]
    weights = np.random.dirichlet(np.ones(len(matrices)), size=1)[0]
    A = sum(w * P for w, P in zip(weights, matrices))
    return A

def kth_roots_of_unity(k):
    """Return the complex k-th roots of unity."""
    return np.array([np.exp(2j * np.pi * i / k) for i in range(k)])

def is_in_convex_hull(z, points):
    """
    Check if complex number z lies in the convex hull of complex points.
    """
    points = np.asarray(points, dtype=np.complex128)
    if len(points.shape) != 1 or len(points) < 2:
        return False  # can't form a convex hull

    num_points = len(points)
    A_eq = np.vstack([
        np.real(points),
        np.imag(points),
        np.ones(num_points)
    ]).T  # shape (num_points, 3) → transpose → (3, num_points)

    b_eq = [np.real(z), np.imag(z), 1.0]
    c = np.zeros(num_points)
    bounds = [(0, 1) for _ in range(num_points)]

    # linprog requires A_eq to be shape (3, num_points) transposed → (num_points, 3)
    if A_eq.shape[1] != len(c):
        return False  # mismatch between vars and equations

    try:
        res = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')
        return res.success
    except ValueError:
        return False



def check_eigenvalue_in_union(z, n):
    """Check if eigenvalue z lies in ⋃_{k=1}^n Π_k."""
    for k in range(1, n + 1):
        if is_in_convex_hull(z, kth_roots_of_unity(k)):
            return True
    return False

def test_eigenvalues(n, trials=10, verbose=True):
    """Run trials to test if eigenvalues of doubly stochastic matrices lie in the conjectured region."""
    violations = []
    for _ in range(trials):
        A = random_doubly_stochastic(n)
        eigvals = eig(A, left=False, right=False)
        for z in eigvals:
            if not np.isclose(z.imag, 0) or not np.isclose(z.real, 1):  # exclude trivial eigenvalue 1
                if not check_eigenvalue_in_union(z, n):
                    violations.append(z)
                    if verbose:
                        print(f"Violation found: eigenvalue {z}")
    if not violations:
        print(f"No violations found for n={n} in {trials} trials.")
    return violations

# Example usage:
violating_eigs = test_eigenvalues(n=6, trials=50)


Violation found: eigenvalue (-0.05564514893868051+0j)
Violation found: eigenvalue (0.0442791939156104+0j)
Violation found: eigenvalue (0.0005403109545555323+0j)
Violation found: eigenvalue (0.00723910012373181+0j)
Violation found: eigenvalue (0.015671948942252954+0j)
Violation found: eigenvalue (0.019385935816385022+0.012414438860818885j)
Violation found: eigenvalue (0.019385935816385022-0.012414438860818885j)
Violation found: eigenvalue (-0.005950012836311221+0.022403144979893215j)
Violation found: eigenvalue (-0.005950012836311221-0.022403144979893215j)
Violation found: eigenvalue (-0.03181565687277145+0j)
Violation found: eigenvalue (0.04291459897373297+0j)
Violation found: eigenvalue (0.023686539345848055+0j)
Violation found: eigenvalue (-0.04353427148719944+0j)
Violation found: eigenvalue (-0.02269415194333719+0.018302611132238168j)
Violation found: eigenvalue (-0.02269415194333719-0.018302611132238168j)
Violation found: eigenvalue (0.01602279370642337+0.023163478140532115j)
Viola